<div style="border:solid orange 3px; padding: 16px">  
    <font size="4">  
        <p style="text-align: center;">
            <b> 1. Основы Python и анализа данных. Исследование предпочтений пользователей Яндекс.Музыки </b>
        </p> 
    </font>
</div>

## 0. Описание проекта 

Имеется таблица с данными о прослушанных композициях в сервисе Яндекс.Музыка. 

Необходимо проанализировать данные и ответить на вопросы:
1. Как музыка, которая звучит в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? 
2. Что и в каком режиме слушают пользователи из Москвы и Петербурга?

## 1. Получение данных
Изучим данные, предоставленные сервисом для проекта.

In [1]:
# импортируем библиотеку pandas
import pandas as pd

path = 'C:/Users/pavel/Desktop/da/projects/datasets/'

In [2]:
# прочтём файл с данными и сохраним его содержимое в df
df = pd.read_csv(path + '01_music_data.csv', index_col=0)

In [3]:
# выведем первые 5 строк таблицы df
df.head(5)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [4]:
# выведем общую информацию о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 4.0+ MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 7 столбцов, тип данных у каждого столбца - строковый.
Рассмотрим, какие в `df` столбцы и какую информацию они содержат:
- `userID` — идентификатор пользователя;
- `Track` — название трека;  
- `artist` — имя исполнителя;
- `genre` — название жанра;
- `City` — город, в котором происходило прослушивание;
- `time` — время, в которое пользователь слушал трек;
- `Day` — день недели.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.

## 2. Предобработка данных
Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименуем столбцы для удобства дальнейшей работы
df.set_axis(['user_id','track_name','artist_name','genre_name',
             'city','time','weekday'], axis = 'columns', inplace = True)

In [7]:
# проверим результат
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

---

In [8]:
# суммарное количество пропусков в таблице
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Обработаем пропуски: заменим пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown', и удалим пропущенные значения в столбце с жанрами.

In [9]:
df['track_name'] = df['track_name'].fillna('unknown')
df['artist_name'] = df['artist_name'].fillna('unknown')

df.dropna(subset = ['genre_name'], inplace = True)

In [10]:
# проверим количество пропусков
df.isnull().sum()

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

---

In [11]:
# суммарное количество дубликатов в таблице
df.duplicated().sum()

3755

In [12]:
# удалим все дубликаты из таблицы df 
df = df.drop_duplicates().reset_index(drop = True)

In [13]:
# проверим количество дубликатов
df.duplicated().sum()

0

---

Выполним поиск неявных дубликатов в столбце с жанрами.

Сохраним список уникальных значений столбца с жанрами в переменной `genres_list`.

In [14]:
genres_list = df.genre_name.unique()
genres_list

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'alternative', 'children', 'rnb', 'hip', 'jazz',
       'postrock', 'latin', 'classical', 'metal', 'reggae', 'tatar',
       'blues', 'instrumental', 'rusrock', 'dnb', 'türk', 'post',
       'country', 'psychedelic', 'conjazz', 'indie', 'posthardcore',
       'local', 'avantgarde', 'punk', 'videogame', 'techno', 'house',
       'christmas', 'melodic', 'caucasian', 'reggaeton', 'soundtrack',
       'singer', 'ska', 'shanson', 'ambient', 'film', 'western', 'rap',
       'beats', "hard'n'heavy", 'progmetal', 'minimal', 'contemporary',
       'new', 'soul', 'holiday', 'german', 'tropical', 'fairytail',
       'spiritual', 'urban', 'gospel', 'nujazz', 'folkmetal', 'trance',
       'miscellaneous', 'anime', 'hardcore', 'progressive', 'chanson',
       'numetal', 'vocal', 'estrada', 'russian', 'classicmetal',
       'dubstep', 'club', 'deep', 'southern', 'black', 'folkrock',
       'fitness', 'french', 'd

In [15]:
# зададим функцию find_genre() для поиска заданной строки в списке жанров и 
# вычисления количества совпадений

def find_genre(need_genre):
    index = 0
    for a in genres_list:
        if  a == need_genre:
            index = index + 1
    return index

Вызовем функцию `find_genre()` для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — `hiphop`. Поищем другие варианты:

- hip
- hop
- hip-hop

In [16]:
find_genre('hip')

1

In [17]:
find_genre('hop')

0

In [18]:
find_genre('hip-hop')

0

In [19]:
# зададим функцию find_hip_hop() для замены неправильного жанра на 'hiphop' 
# в датасете df и проверки результата выполнения

def find_hip_hop(df, wrong_genre_name):
    df['genre_name'] = df['genre_name'].replace(wrong_genre_name, 'hiphop')
    index = df.loc[df.loc[:,'genre_name'] == wrong_genre_name]['genre_name'].count()
    return index

In [20]:
# заменим один неправильный вариант указания жанра 'hip' на 'hiphop'
find_hip_hop(df,'hip')

0

In [21]:
# выведем общую информацию о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
user_id        60126 non-null object
track_name     60126 non-null object
artist_name    60126 non-null object
genre_name     60126 non-null object
city           60126 non-null object
time           60126 non-null object
weekday        60126 non-null object
dtypes: object(7)
memory usage: 3.2+ MB


В таблице отсутствуют пропущенные значения.

## 3. Влияние дня недели на активность пользователей
Было сделано предположение о влиянии дня недели на количество прослушанных композиций. Рассмотрим данные раздельно по двум городам: Москве и Санкт-Петербургу.

In [22]:
# сгруппируем по городу и вычислим общее количество прослушанных композиций
df.groupby('city')['genre_name'].count()

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Петербурге, возможно потому что у Яндекс.Музыки в целом больше пользователей в Москве.

In [23]:
# сгруппируем по дню недели и вычислим общее количество прослушанных композиций
df.groupby('weekday')['genre_name'].count()

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Больше всего прослушиваний в понедельник и пятницу.

In [24]:
# зададим функцию для вычисления количества прослушанных композиций в указанном 
# городе в указанный день недели

def number_tracks(df, day, city):
    track_list = df.loc[(df.loc[:,'weekday'] == day) & (df.loc[:, 'city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count 

In [25]:
# количество прослушиваний для Москвы в понедельник
number_tracks(df, 'Monday', 'Moscow')

15347

In [26]:
# количество прослушиваний для Санкт-Петербурга в понедельник
number_tracks(df, 'Monday', 'Saint-Petersburg')

5519

In [27]:
# количество прослушиваний для Москвы в среду
number_tracks(df, 'Wednesday', 'Moscow')

10865

In [28]:
# количество прослушиваний для Санкт-Петербурга в среду
number_tracks(df, 'Wednesday', 'Saint-Petersburg')

6913

In [29]:
# количество прослушиваний для Москвы в пятницу
number_tracks(df, 'Friday', 'Moscow')

15680

In [30]:
# количество прослушиваний для Санкт-Петербурга в пятницу
number_tracks(df, 'Friday', 'Saint-Petersburg')

5802

Сведём полученную информацию в одну таблицу, где `city`, `monday`, `wednesday`, `friday` названия столбцов.

In [31]:
set0 = [ ['Moscow', 15347, 10865, 15680],
         ['Saint-Petersburg', 5519, 6913, 5802] ]

table = pd.DataFrame(data = set0, columns = ['city', 'monday', 'wednesday', 'friday'])
table

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


Видим, что количество прослушиваний по дням недели различается для разных городов: в Москве наибольшая активность в понедельник и пятницу, в Санкт-Петербурге - наоборот, в среду.

## 4. Влияние времени суток на популярные жанры
Было сделано предположение о влиянии дня недели и времени суток на наиболее популярный жанр музыки. Рассмотрим данные раздельно по двум городам: Москве и Санкт-Петербургу для утра понедельника и вечера пятницы.

Получим таблицы данных по Москве moscow_general и Санкт-Петербургу spb_general.

In [32]:
# получим таблицу moscow_general из тех строк таблицы df, для которых значение
# в столбце 'city' равно 'Moscow'

moscow_general = df.loc[df.loc[:,'city'] == 'Moscow']
moscow_general.head()

,user_id,track_name,artist_name,genre_name,city,time,weekday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,unknown,ruspop,Moscow,09:17:40,Friday


In [33]:
# получим таблицу spb_general
spb_general = df.loc[df.loc[:,'city'] == 'Saint-Petersburg']
spb_general.head()

,user_id,track_name,artist_name,genre_name,city,time,weekday
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
9,E772D5C0,Pessimist,unknown,dance,Saint-Petersburg,21:20:49,Wednesday


In [34]:
# зададим функцию genre_weekday() для определения самых популярных жанров 
# в указанном городе в заданный день и период времени

def genre_weekday(df, day, time1, time2):
    genre_list = df.loc[(df.loc[:,'weekday'] == day) & 
                          (df.loc[:,'time'] > time1) & 
                          (df.loc[:,'time'] < time2)]  
    genre_list_group = genre_list.groupby('genre_name')['genre_name'].count()
    genre_list_sorted = genre_list_group.sort_values(ascending = False).head(10)
    return genre_list_sorted

Сравним самые популярные жанры в Москве и Петербурге для утра понедельника (с 7 до 11) и вечера пятницы (с 17 до 23).

In [35]:
# определим популярные жанры для Москвы в утро понедельника
genre_weekday(moscow_general, 'Monday', '07:00:00','11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: genre_name, dtype: int64

In [36]:
# определим популярные жанры для Петербурга в утро понедельника
genre_weekday(spb_general, 'Monday', '07:00:00','11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre_name, dtype: int64

In [37]:
# определим популярные жанры для Москвы вечером в птяницу
genre_weekday(moscow_general, 'Friday', '17:00:00','23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre_name, dtype: int64

In [38]:
# определим популярные жанры для Петербурга вечером в птяницу
genre_weekday(spb_general, 'Friday', '17:00:00','23:00:00')

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre_name, dtype: int64

Самым популярным жанром (с наибольшим количеством прослушанных композиций) независимо от города и времени прослушивания является популярная музыка. Топ-5 в целом не отличается для обоих городов. 

В топ-10 небольшие отличия, утром в понедельник в Москве популярен жанр `world`, в Петербурге - `jazz`. 

Вечером в пятницу топ-10 популярных жанров музыки в Москве остается без изменений, а в Петербурге вместо `ruspop` появляется `world`. 

## 5. Выявление популярных жанров в разных городах
Было сделано предположение о том, что в Петербурге популярна рэп-музыка, а в Москве - поп-музыка.

Сгруппируем таблицы с композициями по городам по жанру, сосчитаем численность композиций каждого жанра методом count(), отсортируем в порядке убывания и сохраним результат.

In [39]:
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count()\
                .sort_values(ascending = False)

moscow_genres.head(10)

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64

In [40]:
spb_genres = spb_general.groupby('genre_name')['genre_name'].count()\
                .sort_values(ascending = False)

spb_genres.head(10)

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64

Как видим, наиболее популярные жанры совпадают в обоих городах. Рэп-музыка примерно одинаково популярна, а самым часто прослушиваемым жанром является поп-музыка.

## 6. Выводы
Вкусы пользователей Москвы и Петербурга похожи - везде преобладает популярная музыка. В течение недели жанры с наибольшим количеством прослушиваний меняются незначительно в обоих городах. В Москве наибольшую активность пользователи проявляют в понедельник и пятницу, в Санкт-Петербурге - в среду. 